# Sentiment Prediction of Tripadvisor Hotel Reviews
* Inspired by https://www.kaggle.com/shahraizanwar/hotel-reviews-sentiment-prediction
* Using pandas and TensorFlow


In [62]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [53]:
data = pd.read_csv('https://raw.githubusercontent.com/innovad/ml-playground/main/tripadvisor_hotel_reviews.csv')
data.head()
print('Number of lines: ' + str(len(data)))
print('Possible rating numbers: ', end='')
print(data['Rating'].unique())
print ('Max words in Review: ', end='') 
max(data["Review"].apply(lambda x: len(str(x).split(' '))))

Number of lines: 20491
Possible rating numbers: [4 2 3 5 1]
Max words in Review: 

1933

In [63]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["Review"])

X = tokenizer.texts_to_sequences(data["Review"])

max_length = max([len(x) for x in X])
vocab_size = len(tokenizer.word_index)+1
exp_sen = 1

print("Vocabulary size: {}".format(vocab_size))
print("max length of sentence: {}".format(max_length))
print("\nExample:\n")
print("Sentence:\n{}".format(data["Review"][exp_sen]))
print("\nAfter tokenizing :\n{}".format(X[exp_sen]))

X = pad_sequences(X, padding='post', maxlen=350)
print("\nAfter padding :\n{}".format(X[exp_sen]))

Vocabulary size: 52212
max length of sentence: 2012

Example:

Sentence:
ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle, start booked suite paid extra website description not, suite bedroom bathroom standard hotel room, took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast, got kidding, embassy suits sitting room bathroom bedroom unlike kimpton calls suite, 5 day stay offer correct false advertising, send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution, send email trip guest survey did not follow email mail, guess tell concerned guest.the staff ranged indifferent not helpful, asked desk good breakfast spots neighborhood hood told no hotels, gee best breakfast spots seattle 1/2 block away convenient h